# Load necessary packages

In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
pip install scipy

In [ ]:
pip install statsmodels

In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [ ]:
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
# easily detect the differences between different treatments
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.multicomp as mc

# Read Data 

In [ ]:
df = pd.read_csv("esoph.csv")
display(df)

# Visualice Data

In [ ]:
ax = sns.boxplot(x='alcgp', y='cases_per_population', data=df, color='#99c2a2')
ax = sns.swarmplot(x="alcgp", y="cases_per_population", data=df, color='#7d0013')
plt.show()

In [ ]:
ax = sns.boxplot(x='tobgp', y='cases_per_population', data=df, color='#99c2a2')
ax = sns.swarmplot(x="tobgp", y="cases_per_population", data=df, color='#7d0013')
plt.show()

# Fit the Model

In [ ]:
model = ols('cases_per_population ~ tobgp + alcgp + tobgp:alcgp', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

**Interpration**: As we obtain **P-value < 0.05** we can conclude that it existis significative difference among groups. Note that the **F-value** is inverse proportional to the **P-value**

**Warning**: We can only conclude that at least **only one group** is significative difference from the rest, but wa cannot afirm that exists significative difference between the groups

# Check ANOVA assumptions

## Normality of the Residues

In [ ]:
res = model.resid

In [ ]:
sm.qqplot(res, line='45')
plt.xlabel("Theoretical Quantiles")
plt.ylabel("Standardized Residuals")
plt.show()

In [ ]:
plt.hist(res, bins='auto', histtype='bar', ec='k') 
plt.xlabel("Residuals")
plt.ylabel('Frequency')
plt.show()

In [ ]:
stats.shapiro(model.resid)

## Homoscedasticity between Groups

In [ ]:
stats.levene(df['cases_per_population'][df['alcgp'] == '0-39g/day'],
             df['cases_per_population'][df['alcgp'] == '40-79'],
             df['cases_per_population'][df['alcgp'] == '80-119'],
             df['cases_per_population'][df['alcgp'] == '120+'])

In [ ]:
stats.levene(df['cases_per_population'][df['tobgp'] == '0-9g/day'],
             df['cases_per_population'][df['tobgp'] == '10-19'],
             df['cases_per_population'][df['tobgp'] == '20-29'],
             df['cases_per_population'][df['tobgp'] == '30+'])

# Post Hoc Analysis

In [ ]:
comp = mc.MultiComparison(df['cases_per_population'],df['alcgp'])
post_hoc_res = comp.tukeyhsd()
print(post_hoc_res.summary())

In [ ]:
comp = mc.MultiComparison(df['cases_per_population'],df['tobgp'])
post_hoc_res = comp.tukeyhsd()
print(post_hoc_res.summary())

# Exercise 1: Repeat the Two Ways ANOVA test for the tobaco consum (tobgp) with population age (agegp)

In [ ]:
#model = ols().fit()
#anova_table = sm.stats.anova_lm()
#anova_table

In [ ]:
#res = Acces to the residuals

In [ ]:
#### Plot the Q-Q Plot

In [ ]:
#### Plot the Histogram

In [ ]:
#stats.shapiro()

In [ ]:
#stats.levene()